In [19]:
# load module and pre generate data
from graph_tree_module import node, sim, graph_tree

import pickle

# Open the file in binary read mode and load the object
with open("graph_tree.pkl", "rb") as file:
    gt = pickle.load(file)


In [20]:
sliders_data = [{"id": gt.attr_name[i], "min": gt.X_maxmin[1][i], "max": gt.X_maxmin[0][i], "value": gt.root.center[i], "type": "f"} for i in range(len(gt.attr_name))]
# print(sliders_data)

recom_data = [{"i": i, "id": gt.attr_name[i], "max_val": gt.recom[i][0].center[i], "min_val": gt.recom[i][1].center[i]} for i in range(len(gt.attr_name))]
# print(recom_data)

cur = gt.root.down_lv[0]

def get_suround_node_data(node):
    cur_cluster = [{"id": node.id, "position": "center", "x": 300, "y": 200, "color": "red"}]

    contra_cluster = [{"id": -1, "position": "left", "x": 100, "y": 200, "color": "green"}]

    super_cluster = [{"id": node.up_lv.id, "position": "top", "x": 300, "y": 100, "color": "dark red"}] if node.up_lv else []

    foo_sim_node = node.up_lv.down_lv if node.up_lv else []
    sim_cluster = [{"id": foo_sim_node[i].id, "position": "right", "x": 500, "y": 200+i*100, "color": "blue"} for i in range(len(foo_sim_node)) if foo_sim_node[i].id != cur.id] if foo_sim_node else []

    foo_sub_node = node.down_lv if node.down_lv else []
    sub_cluster = [{"id": foo_sub_node[i].id, "position": "bottom", "x": 200+i*100, "y": 350, "color": "pink"} for i in range(len(foo_sub_node))] if foo_sub_node else []

    return cur_cluster + contra_cluster + super_cluster + sim_cluster + sub_cluster

graph_data = get_suround_node_data(cur)
print(graph_data)

[{'id': 1, 'position': 'center', 'x': 300, 'y': 200, 'color': 'red'}, {'id': -1, 'position': 'left', 'x': 100, 'y': 200, 'color': 'green'}, {'id': 0, 'position': 'top', 'x': 300, 'y': 100, 'color': 'dark red'}, {'id': 2, 'position': 'right', 'x': 500, 'y': 300, 'color': 'blue'}, {'id': 3, 'position': 'bottom', 'x': 200, 'y': 350, 'color': 'pink'}, {'id': 4, 'position': 'bottom', 'x': 300, 'y': 350, 'color': 'pink'}]


In [21]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

# slider ####################################################################
@app.route('/sliders')
def sliders():
    return jsonify(sliders_data)

@app.route('/handle_slider', methods=['POST'])
def submit_slider_values():
    data = request.json
    print('Received slider values:', data)
    # Process the data as needed
    return jsonify({'status': 'slider respon success', 'received': data})
# slider end ################################################################

# recom ####################################################################
@app.route('/recom')
def recom():
    return jsonify(recom_data)

@app.route('/handle_recom', methods=['POST'])
def handle_recom():
    data = request.json
    button_id = int(data.get('buttonId'))
    val = int(data.get('val'))
    print(button_id, val)
    moved_node = gt.recom[button_id][val]
    graph_data = get_suround_node_data(moved_node)
    print(graph_data)
    # Handle the button click based on the button_id
    return jsonify({'status': 'recom respon success', 'buttonId': button_id, 'val': val})
# recom end ####################################################################

@app.route('/graph-data')
def graph():
    return jsonify(graph_data)

@app.route('/tree-data')
def tree_data():
    tree = {
        "name": "Root",
        "children": [
            {
                "name": "Child 1",
                "children": [
                    {"name": "Grandchild 1"},
                    {"name": "Grandchild 2"}
                ]
            },
            {
                "name": "Child 2",
                "children": [
                    {"name": "Grandchild 3"},
                    {"name": "Grandchild 4"}
                ]
            }
        ]
    }
    return jsonify(tree)

if __name__ == '__main__':
    app.run(port=8123)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8123
Press CTRL+C to quit
127.0.0.1 - - [02/Sep/2024 17:55:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2024 17:55:27] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [02/Sep/2024 17:55:27] "GET /static/scripts.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2024 17:55:27] "GET /sliders HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2024 17:55:27] "GET /recom HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2024 17:55:27] "GET /tree-data HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2024 17:55:37] "POST /handle_recom HTTP/1.1" 200 -


4 0
[{'id': 52, 'position': 'center', 'x': 300, 'y': 200, 'color': 'red'}, {'id': -1, 'position': 'left', 'x': 100, 'y': 200, 'color': 'green'}, {'id': 25, 'position': 'top', 'x': 300, 'y': 100, 'color': 'dark red'}, {'id': 51, 'position': 'right', 'x': 500, 'y': 200, 'color': 'blue'}, {'id': 52, 'position': 'right', 'x': 500, 'y': 300, 'color': 'blue'}, {'id': 53, 'position': 'right', 'x': 500, 'y': 400, 'color': 'blue'}, {'id': 54, 'position': 'right', 'x': 500, 'y': 500, 'color': 'blue'}]


127.0.0.1 - - [02/Sep/2024 17:56:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2024 17:56:51] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [02/Sep/2024 17:56:51] "GET /static/scripts.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2024 17:56:51] "GET /sliders HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2024 17:56:51] "GET /recom HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2024 17:56:51] "GET /tree-data HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2024 17:56:54] "POST /handle_recom HTTP/1.1" 200 -


3 1
[{'id': 49, 'position': 'center', 'x': 300, 'y': 200, 'color': 'red'}, {'id': -1, 'position': 'left', 'x': 100, 'y': 200, 'color': 'green'}, {'id': 26, 'position': 'top', 'x': 300, 'y': 100, 'color': 'dark red'}, {'id': 49, 'position': 'right', 'x': 500, 'y': 200, 'color': 'blue'}, {'id': 50, 'position': 'right', 'x': 500, 'y': 300, 'color': 'blue'}, {'id': 48, 'position': 'right', 'x': 500, 'y': 400, 'color': 'blue'}]


127.0.0.1 - - [02/Sep/2024 17:58:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2024 17:58:30] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [02/Sep/2024 17:58:30] "GET /static/scripts.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2024 17:58:30] "GET /sliders HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2024 17:58:30] "GET /recom HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2024 17:58:30] "GET /tree-data HTTP/1.1" 200 -
